In [6]:
import pandas as pd
from torchvision.datasets import Caltech256


caltech256_labels = Caltech256(root="datasets/caltech256", download=False).categories
caltech256_targets = pd.read_csv("output/caltech256.csv")

In [7]:
from synthetic_taxonomy import SyntheticTaxonomy


synthetic_taxonomy = SyntheticTaxonomy(
    num_atomic_concepts=257,
    num_domains=2,
    domain_class_count_mean=180,
    domain_class_count_variance=10,
    concept_cluster_size_mean=3,
    concept_cluster_size_variance=1,
)

domain_A = synthetic_taxonomy.domains[0].to_mapping()
domain_B = synthetic_taxonomy.domains[1].to_mapping()

In [8]:
import torch
from models import ResNetMappedClassModel
from datasets import Caltech256MappedClassDataModule
from lightning.pytorch import Trainer


torch.set_float32_matmul_precision("medium")


dataset_domain_A = Caltech256MappedClassDataModule(mapping=domain_A)
model_domain_A = ResNetMappedClassModel.load_from_checkpoint(
    "checkpoints/resnet50-caltech256-synthetic-1-min-val-loss.ckpt"
)
model_domain_A.eval()

dataset_domain_B = Caltech256MappedClassDataModule(mapping=domain_B)
model_domain_B = ResNetMappedClassModel.load_from_checkpoint(
    "checkpoints/resnet50-caltech256-synthetic-2-min-val-loss.ckpt"
)
model_domain_B.eval()

trainer = Trainer(logger=False, enable_checkpointing=False)
model_A_on_domain_B = trainer.predict(model_domain_A, datamodule=dataset_domain_B)
model_B_on_domain_A = trainer.predict(model_domain_B, datamodule=dataset_domain_A)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 33/33 [00:08<00:00,  3.95it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting DataLoader 0: 100%|██████████| 35/35 [00:08<00:00,  4.24it/s]


In [9]:
predictions_A_on_B = torch.cat(model_A_on_domain_B).argmax(dim=1)  # type: ignore
predictions_B_on_A = torch.cat(model_B_on_domain_A).argmax(dim=1)  # type: ignore

domain_A_targets = torch.cat(
    [label for _, label in dataset_domain_A.predict_dataloader()]
)
domain_A_targets = domain_A_targets.map_(
    torch.zeros_like(domain_A_targets), lambda x, _: domain_A[x]
)
domain_B_targets = torch.cat(
    [label for _, label in dataset_domain_B.predict_dataloader()]
)
domain_B_targets = domain_B_targets.map_(
    torch.zeros_like(domain_B_targets), lambda x, _: domain_B[x]
)

In [11]:
df = pd.DataFrame(
    {
        "domain_A": domain_A_targets,
        "predictions_B_on_A": predictions_B_on_A,
    }
)
df.to_csv("output/caltech256_domain_A_predictions.csv", index=False)
df = pd.DataFrame(
    {
        "domain_B": domain_B_targets,
        "predictions_A_on_B": predictions_A_on_B,
    }
)
df.to_csv("output/caltech256_domain_B_predictions.csv", index=False)